In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from IPython.display import Image
np.set_printoptions(precision = 3)

In [2]:
data = pd.read_csv("jester-data-1.csv",header = None)
d = data.to_latex()
text_file = open("Output.txt", "w")
text_file.write(d)
text_file.close()

In [7]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [8]:
data.shape

(24983, 101)

In [9]:
alpha = 0.001
iterations = 30
k = 5
n_features = 2

In [10]:
new_data = data[data.applymap(lambda x: x!=99)]

In [11]:
data_narray = np.array(new_data)

In [17]:
train_indices = np.argwhere(~np.isnan(data_narray))
null_indices = np.argwhere(np.isnan(data_narray))

In [18]:
train_indices

array([[    0,     0],
       [    0,     1],
       [    0,     2],
       ...,
       [24982,    70],
       [24982,    71],
       [24982,    87]], dtype=int64)

In [19]:
user_ratings = new_data.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

In [21]:
print("latent jokes feature size:  ", latent_item_features.shape)
print("latent user feature size:  ", latent_user_preferences.shape)

latent jokes feature size:   (101, 2)
latent user feature size:   (24983, 2)


In [26]:
def predict_rating(user_id,item_id,latent_user_preferences, latent_item_features):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating, latent_user_preferences, latent_item_features, alpha = 0.001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id,latent_user_preferences, latent_item_features)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err, latent_user_preferences, latent_item_features
    



def sgd(fold_train, fold_test,latent_user_preferences, latent_item_features, iterations = 50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    #fold_train= [ list(item) for item in fold_train ]
    #fold_test= [ list(item) for item in fold_test ]
    
    train_mse = 0 
    test_mse = 0
    
    for iteration in range(0,iterations):
        error = []
        for i,j in fold_train:
            user_id = fold_train[i][0]
            item_id = fold_train[i][1]
            rating = user_ratings[i][j]
            train_err,latent_user_preferences, latent_item_features= train(user_id, 
                                                                           item_id, rating, 
                                                                           latent_user_preferences, 
                                                                           latent_item_features, alpha = 0.001)
            error.append(train_err)
       
        train_mse = (np.array(error) ** 2).mean()   
        #if(iteration%10 == 0 ):
         #   print("train mse ",train_mse)
    
    test_error =[]
    for i,j in fold_test:
        user_id = fold_test[i][0]
        item_id = fold_test[i][1]
        rating = user_ratings[i][j]
        err  =user_ratings[i][j]- predict_rating(user_id, item_id,latent_user_preferences, latent_item_features)
        test_error.append(err)
    
    test_mse = (np.array(test_error) ** 2).mean()   
    #print("test mse ",test_mse)
    
    return train_mse, test_mse                